## Kaggle classif driver

In [25]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import cv2
import os
import glob
import datetime
import time

Utile pour voir si la ram tient bon le chargement des images

In [2]:
#import psutil
#psutil.virtual_memory()

###### Fonctions pour charger les images

In [1]:
def get_im(path):
    # Load as grayscale
    img = cv2.imread(path, 0)
    # Reduce size
    resized = cv2.resize(img, (128, 96))
    return resized

def load_train():
    X_train = []
    y_train = []
    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('imgs','train', 'c'+str(j), '*.jpg')
        print path
        files = glob.glob(path)
        for fl in files:
            img = get_im(fl)
            X_train.append(img)
            y_train.append(j)
    return X_train, y_train

def load_test():
    print('Read test images')
    path = os.path.join('imgs', 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        X_test.append(img)
        X_test_id.append(flbase)
        total+=1
        if(total%thr==0):
            print str(total) + " readen images on " + str(len(files)) + "."
    return X_test, X_test_id

Chargement/enregistrement des données <br\>
A décommenter et exécuter uniquement pour charger et enregistrer les images. (Permet ensuite de charge BEAUCOUP plus rapidement)

In [2]:
##############################
##Enregistrement des données##
##############################
'''
X_train, y_train = load_train()

import shelve
d = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/train')
d['X_train'] = X_train
d['y_train'] = y_train
d.close()

del X_train
del y_train

X_test, X_test_id = load_test()

d = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/test')
d['X_test'] = X_test
d['X_test_id'] = X_test_id
d.close()
'''

"\nX_train, y_train = load_train()\n\nimport shelve\nd = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/train')\nd['X_train'] = X_train\nd['y_train'] = y_train\nd.close()\n\ndel X_train\ndel y_train\n\nX_test, X_test_id = load_test()\n\nd = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/test')\nd['X_test'] = X_test\nd['X_test_id'] = X_test_id\nd.close()\n"

#### Quelques fonctions utiles
notamment pour créer la submisson ou d'avoir un train set et test(validation) set

In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

def split_validation_set_with_hold_out(train, target, test_size):
    random_state = 51
    train, X_test, target, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    X_train, X_holdout, y_train, y_holdout = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, X_holdout, y_train, y_test, y_holdout

def create_submission(predictions, test_id, loss):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(round(loss, 6)) + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

def mlogloss(target, pred):
    score=0.0
    for i in range(len(pred)):
        pp = pred[i]
        for j in range(len(pp)):
            prob = pp[j]
            if(prob < 1e-15):
                prob = 1e-15
            score += target[i][j] * math.log(prob)
    return -score/len(pred)

def validate_holdout(model, holdout, target):
    predictions = model.predict(holdout, batch_size=128, verbose=1)
    score = log_loss(target, predictions)
    print('Score log_loss: ', score)
    return score

nest_batch permet d'entrainer par batch nos images, nous pouvons faire varier le nombre d'images que nous donnons en entrée.<br\>
<br\>
oneHotEncoding transforme nos sortie y en onehot (si nous avons des sorties telles que 0,1,2,0 nous aurons maintenant [1,0,0], [0,1,0], [0,0,1], [1,0,0].

In [4]:
def next_batch(X, y, batch_size):
    inds = np.random.choice(len(X), batch_size, replace=False)
    return X[inds], y[inds]

def oneHotEncoding(y, nb_classes):
    res = np.zeros((y.shape[0], nb_classes))
    res[np.arange(y.shape[0]), y] = 1
    return res

## tensorflow
### Building model

In [5]:
import tensorflow as tf

In [7]:
sess = tf.InteractiveSession()

In [8]:
#Nos images sont de tailles 96*128*1
X = tf.placeholder(tf.float32, shape=[None, 96, 128])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [9]:
#Permet de créer les poids et les biais de notre réseau de neurones
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=1e-2)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Facilite la création des convolution et des pooling

In [10]:
# X = input shape 4-D [batch, in_height, in_width, in_channels] (in_channels=number of colors)
# w = filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
def conv2d(X, w):
    return tf.nn.conv2d(X, w, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [11]:
# 1 is the number of input channel, here 1 because it is white and black
# 32 is the number of output channel
# 3 and 3 determine the two dimensions of the patch
w_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])

Nous devons reshape X pour que les entrées soient en dimension 4 et accépter par la fonction de convolution ci-dessus.

In [12]:
X_image = tf.reshape(X, [-1, 96, 128, 1])

In [13]:
h_conv1 = tf.nn.relu(conv2d(X_image, w_conv1) + b_conv1)

In [14]:
w_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])

In [15]:
h_conv2 = tf.nn.relu(conv2d(h_conv1, w_conv2) + b_conv2)
h_pool1 = max_pool_2x2(h_conv2)

# uncomment if we want to normalize the convolution

# h_norm1 = tf.nn.local_response_normalization(h_pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

In [16]:
# Add a dropout
h_pool1_drop = tf.nn.dropout(h_pool1, 0.25)

In [17]:
w_fc1 = weight_variable([48 * 64 * 64, 128])
b_fc1 = bias_variable([128])

Nous redimensionnons afin d'applatir totalement notre image et faire une couche fully connected (lié à la dimension des poids ci-dessus).

In [19]:
h_pool1_drop_flat = tf.reshape(h_pool1_drop, [-1, 48*64*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool1_drop_flat, w_fc1) + b_fc1)

In [20]:
# A new dropout
h_fc1_drop = tf.nn.dropout(h_fc1, 0.5)

In [21]:
w_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

### Training

Il faut maintenant définir le loss, la technique de descente de gradient, l'accuracy

In [22]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

Nous chargeons alors les données d'apprentissage plus tôt enregistrées.

In [23]:
import shelve

d = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/train')
X_train = d['X_train']
y_train = d['y_train']
d.close()

OneHotEncoding pour les sorties et normalisation des pixel pour les X

In [26]:
X_train = np.array(X_train)
y_train = np.array(y_train, dtype=np.uint8)
y_train = oneHotEncoding(y_train, 10)
X_train = X_train.astype('float32')
X_train /= 255

Création d'une base de train et de test (validation) pour controler la descente de gradient.

In [27]:
X_train, X_test, y_test, y_test = split_validation_set(X_train, y_train, 0.2)

Lancement de l'apprentissage (on constate que la descente de gradient ne marche pas très bien)

In [34]:
batch_size = 50
for i in range(200):
    batch = next_batch(X_train, y_train, batch_size)
    if i%10 == 0:
        train_accuracy = accuracy.eval(
            feed_dict={X:batch[0], y_:batch[1]})
        print "step ",str(i)," training accuracy ", str(train_accuracy)
    train_step.run(feed_dict={X:batch[0], y_:batch[1]})

step  0  training accuracy  0.16
step  10  training accuracy  0.2
step  20  training accuracy  0.1
step  30  training accuracy  0.1
step  40  training accuracy  0.12
step  50  training accuracy  0.16
step  60  training accuracy  0.08
step  70  training accuracy  0.12
step  80  training accuracy  0.18
step  90  training accuracy  0.24
step  100  training accuracy  0.14
step  110  training accuracy  0.12
step  120  training accuracy  0.16
step  130  training accuracy  0.1
step  140  training accuracy  0.06
step  150  training accuracy  0.12
step  160  training accuracy  0.16
step  170  training accuracy  0.12
step  180  training accuracy  0.14
step  190  training accuracy  0.1


### Test

In [28]:
# We can delete X_train and y_train now
del X_train, y_train

In [38]:
#############################################
########Chargement des données de test#######
#############################################
import shelve

d = shelve.open('/home/maxence/Documents/kaggle/State_Farm_Distracted_Driver_Detection/imgs/test')
X_test = d['X_test']
X_test_id = d['X_test_id']
d.close()

Je boucle en sélectionneant 100 par 100 les données de test sinon j'ai une erreur de out of memory vu la taille du jeu de données.

In [39]:
j=0
for i in range(100,len(X_test), 100):
    try:
        pred = np.concatenate((pred, y_conv.eval(feed_dict={X:X_test[j:i]})))
    except:
        pred = y_conv.eval(feed_dict={X:X_test[j:i]})
    j=i

if(len(pred)!=len(X_test)):
    pred = np.concatenate((pred, y_conv.eval(feed_dict={X:X_test[i:]})))
del X_test

#### Création du fichier de soumission

In [40]:
create_submission(pred, X_test_id, 999)

In [ ]:
del X_test, X_test_id